In [ ]:
# Eprime data turned into deldisk01 structure.


In [ ]:
import os, datetime
import csv
import sys
import shutil
from openpyxl import load_workbook
import pandas as pd
from io import BytesIO
import numpy as np
import subprocess
from scipy import stats
from ccf.box import LifespanBox
from ccf.config import LoadSettings
from ccf.redcap import RedcapTable 
config = LoadSettings()


In [ ]:
box=LifespanBox(cache='./cache/')
verbose = True
snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')
pathout="./prepped/hcd" 
racepath="./prepped"

#Rosetta (a.k.a Inventory) file will have all the nda vars and pedids
extrainfo=config['rosetta']['filename']

eventlist=['visit_1_arm_1','visit_2_arm_1','visit_3_arm_1','visit_arm_1']

inventory=pd.read_csv(extrainfo)
inventory.columns
inventory=inventory.loc[inventory.nda_age.isnull()==False]
inventory.nda_interview_date=pd.to_datetime(inventory.nda_interview_date).dt.strftime('%m/%d/%Y')
inventory.nda_age=inventory.nda_age.round(0).astype(int)

Fullinventory=inventory.copy()
print("Full Inventory w All events:",Fullinventory.shape)

#
#
#redcapconfigfile="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/.boxApp/redcapconfig.csv"
#verbose = True
#snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')
#pathout="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/prepped_singleton_structures"###

#ndar fields
#extrainfo='UnrelatedHCAHCD_w_STG_Image_and_pseudo_GUID12_11_2020.csv'
#
#box_temp='/.' #location of local copy of curated data
#box = LifespanBox(cache=box_temp)
#e_latestsnapshot=662309868387
cleanestdata = 911854120273


In [ ]:
#get Eprime Data and merge with NDAR required fields
e=box.download_files([cleanestdata])
esnap=pd.read_csv(e[0])
esnap=esnap.loc[esnap.exclude==0].copy()
esnap.head()


In [ ]:
v1ndar=Fullinventory.loc[Fullinventory.redcap_event=='V1'][['nda_age', 'nda_interview_date', 'pseudo_guid','subject', 'M/F','redcap_event']]
v1ndar.shape
v1ndar.columns
esnap.columns

In [ ]:
edata=pd.merge(esnap.drop(columns=['exclude','filename','source','CC',
                                  ]),v1ndar,on=['subject','redcap_event'],how='inner')#.drop(columns=['subject'])
print(edata.columns)
print(len(edata.subject.unique()))
edata=edata.rename(columns={'nda_age':'nda_interview_age','M/F':'nda_gender','redcap_event':'visit','subject':'src_subject_id','pseudo_guid':'subjectkey'})
#rename columns to internal standards (then rename again for nda)
edata.columns
edata.columns=['visit', 'src_subject_id','DDISC.SV_1wk_20', 'DDISC.SV_2wk_20', 'DDISC.SV_1mo_20', 'DDISC.SV_6mo_20', 'DDISC.SV_1yr_20', 'DDISC.SV_3yr_20',
       'DDISC.SV_1wk_100', 'DDISC.SV_2wk_100', 'DDISC.SV_1mo_100', 'DDISC.SV_6mo_100', 'DDISC.SV_1yr_100', 'DDISC.SV_3yr_100',
       'Option Counted', 'MTvMT', 'MvM', 'TvT', 'totalMTvMT', 'totalMvM','totalTvT','CompareMoney',
       'CompareMoneyDelay', 'CompareDelay', 'nda_interview_age',
       'nda_interview_date', 'subjectkey', 'nda_gender']
print(edata.columns)

In [ ]:
#calculate AUCs per email attachment from Greg Burgess, 5/15/20
edata['DDISC.AUC_20'] = (((0.0064 - 0.0000) * (edata['DDISC.SV_1wk_20'] + 20             ) / 2) + \
                ((0.0128 - 0.0064) * (edata['DDISC.SV_2wk_20'] + edata['DDISC.SV_1wk_20']) / 2) + \
                ((0.0278 - 0.0128) * (edata['DDISC.SV_1mo_20'] + edata['DDISC.SV_2wk_20']) / 2) + \
                ((0.1667 - 0.0278) * (edata['DDISC.SV_6mo_20'] + edata['DDISC.SV_1mo_20']) / 2) + \
                ((0.3333 - 0.1667) * (edata['DDISC.SV_1yr_20'] + edata['DDISC.SV_6mo_20']) / 2) + \
                ((1.0000 - 0.3333) * (edata['DDISC.SV_3yr_20'] + edata['DDISC.SV_1yr_20']) / 2)) / 20

edata['DDISC.AUC_100'] = (((0.0064 - 0.0000) * (edata['DDISC.SV_1wk_100'] + 100             ) / 2) + \
                 ((0.0128 - 0.0064) * (edata['DDISC.SV_2wk_100'] + edata['DDISC.SV_1wk_100']) / 2) + \
                 ((0.0278 - 0.0128) * (edata['DDISC.SV_1mo_100'] + edata['DDISC.SV_2wk_100']) / 2) + \
                 ((0.1667 - 0.0278) * (edata['DDISC.SV_6mo_100'] + edata['DDISC.SV_1mo_100']) / 2) + \
                 ((0.3333 - 0.1667) * (edata['DDISC.SV_1yr_100'] + edata['DDISC.SV_6mo_100']) / 2) + \
                 ((1.0000 - 0.3333) * (edata['DDISC.SV_3yr_100'] + edata['DDISC.SV_1yr_100']) / 2)) / 100
edata.head()

In [ ]:
# Add mean accuracy measures for 'catch' questions from kid_eprime measure after rewrite of grab from box
# Derive accuracy from tally of 'EndingQuestionsSlide.RESP' answers for each 'LogFrameStart' in eprime text file.
# Correct response for 'MvM' is 'Option2Chosen'. 
# Correct response for 'MTvMT' and 'TvT' should be 'Option1Chosen'.
# MvM, MTvMT, and TvT are each number of times 'Option1' was chosen (4,2,and 6 trials respectively).  
# total* are the total number of trials (4,2,6) counted explicitly in case of trial interruption. 
# so accuracy when option1 is correct is TvT/totalTvT (=6).  
# invert accuracy if option 2 was correct, as in MvM
edata['DDISC.CompareMoney']         = 1 - (edata['MvM'] / edata['totalMvM']) #always tallied Option1, so need 1-accuracy since correct option for MvM is option 2.  
edata['DDISC.CompareMoneyDelay']= edata['MTvMT'] / edata['totalMTvMT']
edata['DDISC.CompareDelay']         = edata['TvT'] / edata['totalTvT']

In [ ]:
#edata[['Option Counted', 'MvM', 'MTvMT', 'TvT','compare_money','compare_money_at_delay','compare_delay']]

In [ ]:
#map
edata['version_form']="DELAY_3.5"
edata['comqother']="subject about self"
edata=edata.rename(columns={'DDISC.SV_1wk_20':'ddisc_sv_1wk_20',
 'DDISC.SV_2wk_20':'ddisc_sv_2wk_20',
 'DDISC.SV_1mo_20':'ddisc_sv_1mo_20',
 'DDISC.SV_6mo_20':'ddisc_sv_6mo_20',
 'DDISC.SV_1yr_20':'ddisc_sv_1yr_20',
 'DDISC.SV_3yr_20':'ddisc_sv_3yr_20',
 'DDISC.SV_1wk_100':'ddisc_sv_1wk_100',
 'DDISC.SV_2wk_100':'ddisc_sv_2wk_100',
 'DDISC.SV_1mo_100':'ddisc_sv_1mo_100',
 'DDISC.SV_6mo_100':'ddisc_sv_6mo_100',
 'DDISC.SV_1yr_100':'ddisc_sv_1yr_100',
 'DDISC.SV_3yr_100':'ddisc_sv_3yr_100',
 'DDISC.AUC_20':'ddisc_auc_20',
 'DDISC.AUC_100':'ddisc_auc_100',
 'DDISC.CompareMoney':'ddisc_comparemoney',
 'DDISC.CompareMoneyDelay':'ddisc_comparemoneydelay',
 'DDISC.CompareDelay':'ddisc_comparedelay',
 'nda_guid':'subjectkey',
 'subjectped':'src_subject_id',
 'nda_gender':'gender',
 'nda_interview_age':'interview_age',
 'nda_interview_date':'interview_date'                           
                           })
edata.columns

#ddisc_comparemoney DDISC.CompareMoney
#ddisc_comparemoneydelay DDISC.CompareMoneyDelay
#ddisc_comparedelay DDISC.CompareDelay

#reorder columns
edata=edata[['subjectkey', 'src_subject_id', 'gender', 'interview_date','interview_age', 'version_form','comqother',
       'ddisc_sv_1wk_20', 'ddisc_sv_2wk_20', 'ddisc_sv_1mo_20',
       'ddisc_sv_6mo_20', 'ddisc_sv_1yr_20', 'ddisc_sv_3yr_20',
       'ddisc_sv_1wk_100', 'ddisc_sv_2wk_100', 'ddisc_sv_1mo_100',
       'ddisc_sv_6mo_100', 'ddisc_sv_1yr_100', 'ddisc_sv_3yr_100',
       'ddisc_auc_20', 'ddisc_auc_100','ddisc_comparemoney','ddisc_comparemoneydelay','ddisc_comparedelay']]


In [ ]:
edata.columns
#rename gender to sex
edata=edata.rename(columns={'gender':'sex'})

In [ ]:
#Write out structure for validation
filePath=pathout+'/eprime_deldisk01.csv'
if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("deldisk,1\n")
    edata.to_csv(f,index=False)
